In [5]:
%pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 19.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 KB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 5.3 MB/s eta 0:00:0000:0100:01m
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import numpy as np

In [7]:
dataset_df = pd.read_csv("data/cvd_v2.csv", sep=',')

In [8]:
from sklearn.model_selection import train_test_split

X = dataset_df.iloc[:, :-1]
y = dataset_df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 深度学习、用rnn模型，数据需要3通道，在图片中表示RGB，这里表示1
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)


In [10]:
import tensorflow as tf  
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import SimpleRNN, Dense

# 创建模型
'''
该问题本质是二分类问题，故最后一层全连接层用激活函数为：sigmoid
模型结构：
    RNN：隐藏层200，激活函数：relu
    Dense：--> 100(relu) -> 1(sigmoid)
'''
# 创建模型
model = Sequential()
model.add(SimpleRNN(units=200, input_shape=(13, 1), activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()


2025-05-18 16:21:16.301544: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-18 16:21:16.316844: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747556476.335341   37376 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747556476.341607   37376 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747556476.357988   37376 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 200)            │        40,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100)            │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 60,601 (236.72 KB)

 Trainable params: 60,601 (236.72 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-4)

model.compile(
    optimizer=opt,
    loss='binary_crossentropy',  # 二分类问题
    metrics=['accuracy']
)


In [12]:
epochs = 100

history = model.fit(
    X_train, y_train,
    epochs=epochs,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1
)


Epoch 1/100


I0000 00:00:1747556485.400315   37695 service.cc:152] XLA service 0x7f6e940067a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747556485.400349   37695 service.cc:160]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2025-05-18 16:21:25.445288: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1747556485.655453   37695 cuda_dnn.cc:529] Loaded cuDNN version 91001


  45/1932 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6133 - loss: 0.6852

I0000 00:00:1747556486.564977   37695 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1932/1932 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.7061 - loss: 0.5855 - val_accuracy: 0.7238 - val_loss: 0.5570
Epoch 2/100
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.7301 - loss: 0.5472 - val_accuracy: 0.7216 - val_loss: 0.5599
Epoch 3/100
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.7321 - loss: 0.5458 - val_accuracy: 0.7286 - val_loss: 0.5540
Epoch 4/100
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.7355 - loss: 0.5433 - val_accuracy: 0.7243 - val_loss: 0.5594
Epoch 5/100
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.7377 - loss: 0.5382 - val_accuracy: 0.7286 - val_loss: 0.5524
Epoch 6/100
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.7331 - loss: 0.5451 - val_accuracy: 0.7282 - val_loss: 0.5552
Epoch 7/100
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.7376 - loss: 0.5400 - val_accuracy: 0.7275 - val_loss: 0.5511
Epoch 8/100
1932/1932 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.7345 - loss: 0.5412 - va

In [13]:
# 评估：返回的是自己在model.compile中设置，这里为accuracy
score = model.evaluate(X_test, y_test, verbose=0)
print("socre[loss, accuracy]: ", score) # 返回为两个，一个是loss，一个是accuracy


socre[loss, accuracy]:  [0.684647798538208, 0.7118940353393555]
